In [6]:
import sys
import pandas as pd
import numpy as np
import sklearn
import requests

In [7]:
#Read csv data using pandas
#data=pd.read_csv('dataset.csv')

In [26]:
#Length of URL
def URL_len(url):
    return len(url)

#Count of '-'
def count_dash(url):
    dashes = 0
    for char in url:
        if char == '-':
            dashes += 1     
    return dashes

#Count of '@'
def count_at(url):
    at = 0
    for char in url:
        if char == '@':
            at += 1     
    return at

#Count of '?'
def count_q(url):
    q = 0
    for char in url:
        if char == '?':
            q += 1     
    return q

#Count of '%'
def count_percent(url):
    percent = 0
    for char in url:
        if char == '%':
            percent += 1     
    return percent

#Count of '.'
def count_dot(url):
    dot = 0
    for char in url:
        if char == '.':
            dot += 1     
    return dot

#Count of '='
def count_equal(url):
    eq = 0
    for char in url:
        if char == '=':
            eq += 1     
    return eq

#Count of ';'
def count_colon(url):
    colon = 0
    for char in url:
        if char == ';':
            colon += 1     
    return colon

#Count of numbers
def count_numbers(url):
    numbers = sum(c.isdigit() for c in url)
    return numbers

#Count of letters
def count_letters(url):
    letters = sum(c.isalpha() for c in url)
    return letters

#HTTP vs. HTTPS
#Returns 1 if HTTPS, 0 if HTTP
#If inputted URL does not contain either, does a browser request using HTTPS, if an error occurs then is http
def is_http(url):
    if url[:5] == 'https':
        return 1
    elif url[:4] == 'http':
        return 0
    else:
        try:
            r = requests.get('https://'+ url)
            return 1
        except:
            return 0


In [3]:
#Create a dataframe to contain our processsed data with custom columns
processed_data = pd.DataFrame(columns=['url','evil','length','periods','semicolons','etc'])
#To print
processed_data

,url,evil,length,periods,semicolons,etc
